Llamada con 1 parametro

In [3]:
import requests
import base64
import pandas as pd

# Credenciales de la API de Idealista (Reemplaza con tus credenciales)

API_KEY = "qcr3cdfxbnkiwdfx0u9mokearldjejzi"
API_SECRET = "2eDk9ISgSlBT"

# URL de autenticación
TOKEN_URL = "https://api.idealista.com/oauth/token"

# Codificación de credenciales en base64
auth_string = f"{API_KEY}:{API_SECRET}"
auth_bytes = auth_string.encode("utf-8")
auth_base64 = base64.b64encode(auth_bytes).decode("utf-8")

# Petición para obtener el token de acceso
token_headers = {
    "Authorization": f"Basic {auth_base64}",
    "Content-Type": "application/x-www-form-urlencoded;charset=UTF-8"
}
token_data = {
    "grant_type": "client_credentials",
    "scope": "read"
}
token_response = requests.post(TOKEN_URL, headers=token_headers, data=token_data)
token_json = token_response.json()

# Extraer el token de acceso
ACCESS_TOKEN = token_json.get("access_token")

if not ACCESS_TOKEN:
    print("Error obteniendo el token:", token_json)
    exit()

# URL de búsqueda de propiedades
SEARCH_URL = "https://api.idealista.com/3.5/es/search"

# Parámetros de la búsqueda
search_params = {
    "operation": "sale",  # Opciones: "sale" o "rent"
    "propertyType": "homes",  # Opciones: homes, offices, premises, garages, bedrooms
    "center": "41.3851,2.1734",  # Coordenadas de Barcelona (latitud, longitud)
    "distance": 15000,  # Radio en metros (15 km)
    "maxItems": 50,  # Máximo permitido
    "numPage": 1,  # Página inicial
    "sort": "desc",
    "order": "publicationDate",
    "locale": "es"
}

# Headers con el token de autorización
search_headers = {
    "Authorization": f"Bearer {ACCESS_TOKEN}",
    "Content-Type": "application/json"
}

# Realizar la petición de búsqueda
response = requests.post(SEARCH_URL, headers=search_headers, params=search_params)
data = response.json()

# Extraer la lista de propiedades
properties = data.get("elementList", [])

# Si no hay propiedades, mostrar mensaje y salir
if not properties:
    print("No se encontraron propiedades.")
    exit()

# Convertir los datos a un DataFrame de pandas
df = pd.DataFrame(properties)

# Guardar en un archivo CSV
csv_filename = "pisos_barcelona.csv"
df.to_csv(csv_filename, index=False, encoding="utf-8")

print(f"Datos guardados en {csv_filename}")


Datos guardados en pisos_barcelona.csv


Llamada para las siguientes 50 paginas

In [5]:
import requests
import base64
import pandas as pd
import time

# Credenciales de la API de Idealista (Reemplaza con tus credenciales)
#API_KEY = "zvu2itk0xxsv0maxtf0ng2naevde8ypv"
#API_SECRET = "qhMbwJq8Fizn"

API_KEY = "qcr3cdfxbnkiwdfx0u9mokearldjejzi"
API_SECRET = "2eDk9ISgSlBT"

# URL de autenticación
TOKEN_URL = "https://api.idealista.com/oauth/token"

# Codificación de credenciales en base64
auth_string = f"{API_KEY}:{API_SECRET}"
auth_bytes = auth_string.encode("utf-8")
auth_base64 = base64.b64encode(auth_bytes).decode("utf-8")

# Petición para obtener el token de acceso
token_headers = {
    "Authorization": f"Basic {auth_base64}",
    "Content-Type": "application/x-www-form-urlencoded;charset=UTF-8"
}
token_data = {
    "grant_type": "client_credentials",
    "scope": "read"
}
token_response = requests.post(TOKEN_URL, headers=token_headers, data=token_data)
token_json = token_response.json()

# Extraer el token de acceso
ACCESS_TOKEN = token_json.get("access_token")

if not ACCESS_TOKEN:
    print("Error obteniendo el token:", token_json)
    exit()

# URL de búsqueda de propiedades
SEARCH_URL = "https://api.idealista.com/3.5/es/search"

# Parámetros base de la búsqueda
base_params = {
    "operation": "sale",  # Opciones: "sale" o "rent"
    "propertyType": "homes",  # Opciones: homes, offices, premises, garages, bedrooms
    "center": "41.3851,2.1734",  # Coordenadas de Barcelona (latitud, longitud)
    "distance": 15000,  # Radio en metros (15 km)
    "maxItems": 50,  # Máximo permitido por llamada
    "sort": "desc",
    "order": "publicationDate",
    "locale": "es"
}

# Headers con el token de autorización
search_headers = {
    "Authorization": f"Bearer {ACCESS_TOKEN}",
    "Content-Type": "application/json"
}

# Lista para almacenar todas las propiedades
all_properties = []

# Iterar sobre las páginas desde la 2 hasta la 50
for page in range(2, 51):
    print(f"Obteniendo página {page}...")
    base_params["numPage"] = page  # Modificar número de página

    try:
        # Hacer la petición a la API
        response = requests.post(SEARCH_URL, headers=search_headers, params=base_params)
        data = response.json()

        # Verificar si la respuesta contiene propiedades
        properties = data.get("elementList", [])
        if not properties:
            print(f"No se encontraron propiedades en la página {page}, deteniendo el proceso.")
            break

        # Agregar los datos a la lista
        all_properties.extend(properties)

        # Guardar en CSV después de cada llamada para evitar pérdidas en caso de error
        df = pd.DataFrame(all_properties)
        csv_filename = "pisos_barcelona.csv"
        df.to_csv(csv_filename, index=False, encoding="utf-8")
        print(f"Guardado en {csv_filename} con {len(all_properties)} propiedades hasta ahora.")

    except Exception as e:
        print(f"Error en la página {page}: {e}")
        print("Guardando datos obtenidos hasta el momento antes de salir...")
        df = pd.DataFrame(all_properties)
        df.to_csv(csv_filename, index=False, encoding="utf-8")
        print(f"Datos guardados en {csv_filename}. Finalizando ejecución.")
        break  # Detener el bucle en caso de error

    # Respetar el límite de 1 llamada por segundo
    time.sleep(1)

print(f"Proceso completado. Datos finales guardados en {csv_filename}.")


Obteniendo página 2...
Guardado en pisos_barcelona.csv con 50 propiedades hasta ahora.
Obteniendo página 3...
Guardado en pisos_barcelona.csv con 100 propiedades hasta ahora.
Obteniendo página 4...
Guardado en pisos_barcelona.csv con 150 propiedades hasta ahora.
Obteniendo página 5...
Guardado en pisos_barcelona.csv con 200 propiedades hasta ahora.
Obteniendo página 6...
Guardado en pisos_barcelona.csv con 250 propiedades hasta ahora.
Obteniendo página 7...
Guardado en pisos_barcelona.csv con 300 propiedades hasta ahora.
Obteniendo página 8...
Guardado en pisos_barcelona.csv con 350 propiedades hasta ahora.
Obteniendo página 9...
Guardado en pisos_barcelona.csv con 400 propiedades hasta ahora.
Obteniendo página 10...
Guardado en pisos_barcelona.csv con 450 propiedades hasta ahora.
Obteniendo página 11...
Guardado en pisos_barcelona.csv con 500 propiedades hasta ahora.
Obteniendo página 12...
Guardado en pisos_barcelona.csv con 550 propiedades hasta ahora.
Obteniendo página 13...
Guarda